# Evaluación y Uso de YOLO para Detección de Objetos

## Introducción

En este proyecto, exploramos y evaluamos el rendimiento del modelo **YOLO (You Only Look Once)** para la detección de objetos en imágenes. Dado que YOLO utiliza una arquitectura ViT para la detección en imágen devolviendo bounding boxes, es ideal para empezar viendo el funcionamiento de modelos en hugging face

El objetivo principal de este trabajo es comprobar la funcionalidad de YOLO y practicar su implementación en Jupiter Notebook para familiarizarnos con las herramientas

## Objetivos

- Implementar y probar el modelo **YOLO** en imágenes de prueba.
- Entrenar el modelo en un conjunto de datos propuesto y evaluar su rendimiento.
- Practicar con todo ello implementaciones y evaluacion de modelos en Hugging Face

¡Comencemos con la evaluación del modelo!

In [1]:
import os
import cv2
import numpy as np
from enum import Enum
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import YolosForObjectDetection, YolosImageProcessor

**Comprobar el modelo**

Para ello seguiremos los siguientes pasos

1. Elegimos un dataset apropiado
2. Procesamos los datos para cumplir con el formato de entrada del modelo
3. Entrenamos el modelo
4. Evaluamos el rendimiento

### 1 Elegir un dataste apropiado

En este caso usaremos COCO, es un dataset de imágenes muy utilizado y escojemos este ya que es el usado por los propios creadores del modelo para evaluar su rendimiento, por lo que trataremos de imitar su experimento

COCO tiene una estructura particular en la que la información de cada imagen viene en un JSON, los detalles del formato se pueden encontrar [aquí](https://cocodataset.org/#format-data). Pero esencialmente buscamos crear un diccionario con las imagenes, id, bbx y clase para poder cargarlo en el Dataloader

In [2]:
from utils import COCODataProcessor
COCO_DIR = 'datasets/COCO_2017'

coco_data = COCODataProcessor(f'{COCO_DIR}/annotations/instances_train2017.json', 
                              f'{COCO_DIR}/train2017')

print(f"coco train annotations: {coco_data.json['annotations'][0]}")
print(f"coco train images: {coco_data.json['images'][0]}")

coco train annotations: {'segmentation': [[239.97, 260.24, 222.04, 270.49, 199.84, 253.41, 213.5, 227.79, 259.62, 200.46, 274.13, 202.17, 277.55, 210.71, 249.37, 253.41, 237.41, 264.51, 242.54, 261.95, 228.87, 271.34]], 'area': 2765.1486500000005, 'iscrowd': 0, 'image_id': 558840, 'bbox': [199.84, 200.46, 77.71, 70.88], 'category_id': 58, 'id': 156}
coco train images: {'license': 3, 'file_name': '000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}


Una vez tenemos el diccionario con el formato deseado pasamos a convertirlo en un dataset usando als funciones de torch

In [3]:
coco_dataset = coco_data.image_dataset

Como podemos observar tenemos una cantiadad suficiente de datos por lo que ya que este ejercicio es para familiarizarnos con el proceso vamoa a partir de este dataset coger unos splits de train test y validation mucho menores para ver el resultado

In [4]:
coco_dataset.shape

(117266, 3)

In [7]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

TRAIN_SIZE = 10000
VAL_SIZE = 2000
TEST_SIZE = 1000

# Convertimos el dataset en un array de indices de imagenes para hacer el split mas eficientemente
indices = np.arange(len(coco_data.image_dict.keys()))

# Obtenemos la cantidad de imagenes con las que queremos trabajar aleatriamente
selected_indices = np.random.choice(indices, size=TRAIN_SIZE+VAL_SIZE+TEST_SIZE, replace=False)

# Separamos Test del conjunto de imagenes
train_val_indices, test_indices = train_test_split(indices, test_size=TEST_SIZE, random_state=123)

# Separamos validation de train y terminamos con los 3 conjuntos
train_indices, val_indices = train_test_split(train_val_indices, test_size=VAL_SIZE, random_state=123)

# Extraemos los subconjuntos correspondientes a cada división
train_split = {key: coco_data.image_dict[key] for key in train_indices}
val_split = {key: coco_data.image_dict[key] for key in val_indices}
test_split = {key: coco_data.image_dict[key] for key in test_indices}

Una vez creados los splits vamos a crear los tensores con la información en formato de YOLO.

Este modelo utiliza de **input las imágenes** por el contrario de **output** tenemos las **bounding boxes y sus clases de COCO**

In [ ]:
from utils import COCOImageDataset
from torch.utils.data import Dataset, DataLoader

BATCH_SIZE = 10

dtrain = COCOImageDataset(train_split)
dval = COCOImageDataset(val_split)

train_loader = DataLoader(dtrain, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dval, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-base')
processor = YolosImageProcessor.from_pretrained('hustvl/yolos-base')

Ahora podemos pasar a la fase de entrenamiento

In [8]:
model.train()   # ponemos el modelo en modo entrenamiento

NameError: name 'model' is not defined

In [11]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler

# Crea dataloaders
train_loader = DataLoader(dataset["train"], batch_size=16, shuffle=True)
val_loader = DataLoader(dataset["validation"], batch_size=16)

# Configura optimizador y scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader))

# Entrenamiento básico
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):  # Número de épocas
    model.train()
    for batch in train_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["label"].to(device)
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
    print(f"Epoch {epoch + 1} complete!")

NameError: name 'dataset' is not defined